提示词指的是输入到llm的文本，一个准确的提问会让llm的回答更加准确，提示词不仅改变提问内容，还涉及逻辑思维、迭代优化问题，所以有学科（岗位）叫提示词开发工程师

langchain抽象了提示词组件，方便定义、修改提示词

In [4]:
from langchain import prompts
atts=list(filter(lambda att:att.lower().find('prompt')>=0,dir(prompts)))
print(atts)

['AIMessagePromptTemplate', 'BaseChatPromptTemplate', 'BasePromptTemplate', 'ChatMessagePromptTemplate', 'ChatPromptTemplate', 'FewShotChatMessagePromptTemplate', 'FewShotPromptTemplate', 'FewShotPromptWithTemplates', 'HumanMessagePromptTemplate', 'PipelinePromptTemplate', 'Prompt', 'PromptTemplate', 'StringPromptTemplate', 'SystemMessagePromptTemplate', 'load_prompt', 'prompt']


通过查看这个组件的模块，可以看到其中主要有3个基类`BasePromptTemplate`，`BaseChatPromptTemplate`、`PromptValue`，我们分别学习其中代表性的`PromptTemplate`、`ChatPromptTemplate`

```mermaid
classDiagram
    class BasePromptTemplate
    class PipelinePromptTemplate
    class StringPromptTemplate
    class BaseChatPromptTemplate
    class PromptTemplate
    class FewShotPromptTemplate
    class FewShotPromptWithTemplates
    class AutoGPTPrompt
    class ChatPromptTemplate
    class AgentScratchPadChatPromptTemplate
    class BaseMessagePromptTemplate
    class MessagesPlaceholder
    class BaseStringMessagePromptTemplate
    class HumanMessagePromptTemplate
    class AIMessagePromptTemplate
    class SystemMessagePromptTemplate

    PromptValue<|-- StringPromptValue
    PromptValue<|-- ChatPromptValue
                    
    BasePromptTemplate <|-- PipelinePromptTemplate
    BasePromptTemplate <|-- StringPromptTemplate
    BasePromptTemplate <|-- BaseChatPromptTemplate

    StringPromptTemplate <|-- PromptTemplate
    StringPromptTemplate <|-- FewShotPromptTemplate
    StringPromptTemplate <|-- FewShotPromptWithTemplates
    BaseChatPromptTemplate <|-- AutoGPTPrompt
    BaseChatPromptTemplate <|-- ChatPromptTemplate
    ChatPromptTemplate <|-- AgentScratchPadChatPromptTemplate
    BaseMessagePromptTemplate <|-- MessagesPlaceholder
    BaseMessagePromptTemplate <|--                               BaseStringMessagePromptTemplate
    BaseStringMessagePromptTemplate <|-- ChatMessagePromptTemplate
    ChatMessagePromptTemplate <|-- HumanMessagePromptTemplate
    ChatMessagePromptTemplate <|-- AIMessagePromptTemplate
    ChatMessagePromptTemplate <|-- SystemMessagePromptTemplate                                               
```

## 字符串提示模板

In [33]:
from langchain.prompts import PromptTemplate

template="""你是城市{city}的导游，负责带{place}游客去白云山游玩"""
prompt_template = PromptTemplate.from_template(template=template)
prompt=prompt_template.invoke({'city':"广州",'place':"国外"})
prompt

StringPromptValue(text='你是城市广州的导游，负责带国外游客去白云山游玩')

可以看出，提示模板接受一个字典作为输入，其中每个键表示提示模板中要填充的变量,ChatPromptTemplate 类似 ptyhon 的 string.format()，先声明变量位置，然后往里面填值，基于以上内容直接输入llm，可以得到llm答案

In [34]:
from langchain_ollama import OllamaLLM
# 初始化Ollama LLM，注意需要后台开启ollama服务
model_name = "qwen2.5:latest"
model  = OllamaLLM(model=model_name,base_url='http://localhost:11434')
response=model.invoke(prompt)
print(response)

非常高兴能为您服务！作为您的导游，我将带领您探索广州市著名的白云山。现在，请允许我先向您介绍一下白云山的基本情况和一些游览小贴士。

### 白云山概况

- **地理位置**：位于广州市中心偏北位置，东临珠江，西连广州大学城。
- **自然风光**：作为广州市的“绿肺”，白云山拥有丰富的植被，四季皆有不同的景色。春天赏花、夏天避暑、秋天观叶、冬天雪景（虽较少见）都各有特色。
- **人文景观**：除了自然风光外，山上还分布着许多历史遗迹和文化景点。

### 游览路线与行程安排

我们今天将沿着主要的登山步道进行游览。全程大约需要3-4小时左右，具体时间取决于您的体力情况和个人兴趣。

1. **入口进入**
   - 从白云山南门进入（建议选择这一入口，因为这里人流相对较少且风景较佳）。
2. **主干道路线**
   - 走过主要的登山步道，一路上可以欣赏到不同种类的树木和花卉。
3. **重要景点参观**：
   - 沿途会经过摩星岭、鸣春谷等著名景点。您可以在这些地方稍作停留拍照或休息。
4. **返回**
   - 根据您的体力情况和时间安排，可以选择直接下山或者选择乘坐缆车快速返回。

### 注意事项

- 请穿着舒适的鞋子并带上足够的水份补给。
- 夏季记得涂抹防晒霜，并随身携带太阳镜等防晒用品。
- 携带一些简单的零食，在休息时享用。
- 尊重自然，不要乱丢垃圾或破坏植被。

如果您有任何特殊需求或者想要探索特定的地方，请随时告知我。希望这次游览能让您更好地了解广州这座美丽的城市！


## 聊天提示词

以上只是向llm发送一次提问，如果要向llm发送“一段对话”，可以使用`ChatPromptTemplate`

In [35]:
from langchain.prompts import ChatPromptTemplate

# 首先，构造一个提示模版字符串：`template_string`
template_string = """把由三个反引号分隔的文本\
翻译成一种{style}风格。\
文本: ```{text}```
"""

chat_prompt_template = ChatPromptTemplate.from_template(template_string)
chat_prompt=chat_prompt_template.invoke({'style':"普通话",'text':"Hello, Guangzhou!"})
chat_prompt

ChatPromptValue(messages=[HumanMessage(content='把由三个反引号分隔的文本翻译成一种普通话风格。文本: ```Hello, Guangzhou!```\n', additional_kwargs={}, response_metadata={})])

In [36]:
from langchain_ollama import ChatOllama
# 初始化Ollama LLM，注意需要后台开启ollama服务
model_name = "qwen2.5:latest"
chat_model  = ChatOllama(model=model_name,base_url='http://localhost:11434')

response=chat_model.invoke(chat_prompt)
response

AIMessage(content='```你好，广州！```', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2024-12-30T05:54:33.4289242Z', 'done': True, 'done_reason': 'stop', 'total_duration': 607320800, 'load_duration': 58754400, 'prompt_eval_count': 55, 'prompt_eval_duration': 109000000, 'eval_count': 7, 'eval_duration': 436000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-65f06d12-295a-4540-81f2-b267750fc470-0', usage_metadata={'input_tokens': 55, 'output_tokens': 7, 'total_tokens': 62})

注意，这里为了区分人类信息、机器信息，langchian抽象出Messege的概念，并以HumanMessage、AIMessage区分，当准备向llm传递一段对话时，可以进行以下操作

In [28]:
from langchain_core.messages import AIMessage,HumanMessage

messages=[
    HumanMessage(content="你叫小广，专门回答关于广州的问题"),
    AIMessage(content="好的，我叫小广，设计用于回答广州的问题"),
    HumanMessage(content="你是谁？")
]

response=chat_model.invoke(messages)
response

AIMessage(content='我是来自阿里云的AI助手小广，专门为您提供关于广州的相关信息和问题解答。有什么关于广州的问题可以随时问我哦！', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2024-12-30T05:38:50.4394312Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1093494400, 'load_duration': 39523600, 'prompt_eval_count': 64, 'prompt_eval_duration': 112000000, 'eval_count': 30, 'eval_duration': 939000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-59bc1e0b-8a99-4351-bd2d-ede51ba97337-0', usage_metadata={'input_tokens': 64, 'output_tokens': 30, 'total_tokens': 94})

实际上，langchian不仅有`AIMessage`,`HumanMessage`，还有以下Message

In [31]:
from langchain_core import messages
atts=list(filter(lambda att:att.lower().find('message')>=0,dir(messages)))
print(atts)

['AIMessage', 'AIMessageChunk', 'AnyMessage', 'BaseMessage', 'BaseMessageChunk', 'ChatMessage', 'ChatMessageChunk', 'FunctionMessage', 'FunctionMessageChunk', 'HumanMessage', 'HumanMessageChunk', 'MessageLikeRepresentation', 'RemoveMessage', 'SystemMessage', 'SystemMessageChunk', 'ToolMessage', 'ToolMessageChunk', '_message_from_dict', 'convert_to_messages', 'convert_to_openai_messages', 'filter_messages', 'merge_message_runs', 'message_chunk_to_message', 'message_to_dict', 'messages_from_dict', 'messages_to_dict', 'trim_messages']


这段脚本展示了如何使用LangChain库中的提示词组件来构建和调用LLM（大型语言模型），具体包括**字符串提示模板、聊天提示模板以及消息对象**的使用。更复杂的使用包括：使用少样本提示选择器、绑定结构化输出解释器，这将在进阶内容上提出